##### 수입원장을 읽어서 직할지점 convert


In [ ]:
import pandas as pd
import GladUtil as glad
import time
import os

team_dict = {'시흥':'글래드시흥','씨티엠':'글래드씨티엠','HL':'글래드HL','강동':'글래드강동'}
brh_dict = {'엠피인스모아':'인스모아'}
sawon_dict = {'박정호':'글래드강동','허유지':'글래드강동'}

''' 글래드내 기준의 지점으로 컨버전 '''
# 제이엔제이에서 글래드프라임으로 이동건은 지점명을 제이엔제이로 변경-20210701 "
def glad_prime(df_p):
    cond = " 팀 == '프라임2팀' "
    temp = [ '글래드제이엔제이' if x < '20210701' else '글래드프라임' for x in df_p.query(cond)['계약일자'] ]
    cond = df_p['팀'] == '프라임2팀'
    df_p.loc[cond ,'지점'] = temp
    return df_p

def read_source_adv(in_file):
    ''' 원장 read '''
    print('원장 start reading...')
    start = time.time()  # 시작 시간 저장
    dfW = pd.read_excel(in_file, header=0, sheet_name='전체',dtype={'계약일자':object})  
    print(' 원장 read completion ')

    # 직할지점 컬럼 conv
    dfW['팀'] = dfW['직할지점']
    #--변환하지 않고 프라임2팀 명칭으로 예외처리
    # dfW = glad_prime(dfW)
    dfW['팀'].fillna('',inplace=True)

    # 지점
    dfW.loc[:,'지점'].fillna('글래드', inplace=True)  #결측치 보정
    dfW.loc[:,'지점'] = dfW['지점'].apply(lambda x: brh_dict.get(x,x))
    # 팀 -> 김묘정 -> 사원명으로 변환하여 직할지점 컬럼생성
    dfW.loc[:,'직할지점'] = dfW.apply(lambda x: team_dict.get(x['팀'], x['지점']), axis=1)
    dfW.loc[:,'직할지점'] = dfW.apply(lambda x: glad.fcConv(x['직할지점'], x['사원번호'], x['계약일자']), axis=1)	
    dfW.loc[:,'직할지점'] = dfW.apply(lambda x: sawon_dict.get(x['사원명'], x['직할지점']), axis=1)

    dfW['수수료계'].fillna(0, inplace=True)  #결측치 보정
    
    print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
    return dfW


DIR_PATH = 'e:/글로벌백업(180329)/04.수수료작업/'
ym ='202202'
inf_nm = ym + '수수료/'+ ym[0:4]+ ym[4:6]+ '_수입원장.xlsx'
in_file = DIR_PATH + inf_nm                   #원장

dfW = pd.DataFrame()
dfW = read_source_adv(in_file)
# dfW.to_excel(in_file.replace('수입원장','수입원장_지점변환'), index = False)

''' sheet_nm이 지정된 시트가 존재하면 시트를 삭제 후 생성, 없으면 새로 생성'''
sheet_nm = '전체'
w_mode = 'a' if os.path.isfile(in_file) else 'w'  #파일있으면 append mode
with pd.ExcelWriter(in_file, mode = w_mode, engine="openpyxl") as writer:
    workbook = writer.book
    try:
        workbook.remove(workbook[sheet_nm])
    except:
        print('create new sheet')
    finally:
        dfW.to_excel(writer, sheet_name=sheet_nm, index=False)  


# df_s = pd.pivot_table(dfW,values=['수수료계'],index=['지점','팀','사원명'],
    # columns=['계약종류'],fill_value=0,aggfunc=sum).reset_index()
df_s = pd.pivot_table(dfW,values=['수수료계'],index=['지점','팀','사원명','계약종류'],
      fill_value=0,aggfunc=sum).reset_index()       
print('<<<<<--- job completed --->>>>>')


##### IBSS파일 읽기

In [ ]:
ibss_file = []
ibss_file.append('마감실적리스트(장기)_202202240101.xlsx')
ibss_file.append('마감실적리스트(생보)_202202240101.xlsx')
ibss_file.append('마감실적리스트(일반)_202202240101.xlsx')
ibss_file.append('마감실적리스트(자동차)_202202240101.xlsx')

import re
df_all = pd.DataFrame()
for in_file in ibss_file:
    items = re.findall('\(([^)]+)', in_file) #extracts string in bracket() print items
    dfT = pd.read_excel(DIR_PATH  + ym + '수수료/'+in_file, header=0, dtype={'증권번호':object})
    dfT['보종'] = items[0]
    df = pd.pivot_table(dfT,values=['수수료H','수수료S'],index=['지점','팀','담당','보종'],
         fill_value=0,aggfunc=sum).reset_index()
    df_all = pd.concat([df_all,df])

In [ ]:
out_file = 'e:/temp/지점손익.xlsx'

# with pd.ExcelWriter(out_file,engine='openpyxl',if_sheet_exists='overlay') as writer:
#     df_s.to_excel(writer, sheet_name='원장',index=False)
#     df_all.to_excel(writer, sheet_name="수수료", index =False)

# import openpyxl
# book = openpyxl.load_workbook(out_file)
# writer = pd.ExcelWriter(out_file, engine='openpyxl', mode='a',if_sheet_exists='overlay')
# writer.book = book
# writer.sheets = {ws.title: ws for ws in book.worksheets}

# for sheetname in writer.sheets:
#     print(sheetname)
#     if writer.sheets[sheetname] =='원장':
#         df_s.to_excel(writer, sheet_name=sheetname,startrow=writer.sheets[sheetname].max_row, index=False,header=False)
#     elif writer.sheets[sheetname] =='수수료':
#         df_all.to_excel(writer, sheet_name=sheetname,startrow=writer.sheets[sheetname].max_row, index =False)